# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: Direct Preference Optimization 

__Мягкий дедлайн 16.11.25 23:59__ \
__Жесткий дедлайн 19.11.25 23:59__

### О задании

В этом задании вам предстоит обучить большую LLM для ответов на вопросы с помощью DPO, а также реализовать LoRA для эффективного обучения. 

### Оценивание и штрафы

Максимально допустимая оценка за работу — __11 баллов__.

Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета баллы за соответствующие задания не будут ставиться. Мы настаиваем на том, чтобы вы оформили весь код в виде полноценного проекта. Этот ноутбук нужно рассматривать скорее как файл с условием, чем как место для написания массивного кода. За сдачу больших ноутбуков с кодом оценка будет снижена. Ответы на все вопросы в заданиях можно (нужно) писать в отчете.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

### План решения

<img src="https://miro.medium.com/v2/resize:fit:1400/1*lK6iJMz5CGh2fo7TsDn15A.png" alt="drawing" width="700"/>

Обучение следованию инструкциям с помощью DPO разбивается на два этапа:    
1. __Supervised Fine-tuning (SFT)__ – обучение базовой модели ответам на запросы в нужном формате.
2. __Direct Preference Optimization (DPO)__ – обучение SFT модели приоритизации "хороших" ответов.

Мы не хотим обучать модели целиком по двум причинам: 1) используемые модели очень большие; 2) нам требуется лишь выравнить модель с нашими предпочтениями, не внося в нее новых знаний, что не требует серьезного обучения. Поэтому мы будем использовать PEFT, а именно LoRA для обучения.

Таким образом, вам надо будет:
1. Реализовать и протестировать LoRA
2. Разобраться с данными и привести их к нужному формату
3. Обучить SFT модель
4. Обучить DPO модель
5. Порадоваться, что вы молодцы и со всем справились
6. (Опционально) сделать веб-интерфейс для вашей модели, переиспользуя код из первой домашки (мы можем выдать бонусы, если получится классно).

### О датасете

Мы будем работать с датасетом [Anthropic Helpful-Harmless](https://huggingface.co/datasets/Anthropic/hh-rlhf) для RLHF. В нем содержится 160к примеров ответов на вопросы с историей.

### Low-Rank Adaptation (LoRA)

<img src="https://heidloff.net/assets/img/2023/08/lora.png" alt="drawing" width="600"/>

__Задание 1 (3 балла).__ Реализуйте самостоятельно модуль LoRA для эффективного обучения LLM по схеме, описанной в [статье](https://arxiv.org/pdf/2106.09685). Встройте его в свою любимую LLM и убедитесь, что ошибка убывает при обучении параметров LoRA на безусловную генерацию. Для этого возьмите любые данные на свой выбор. Замерьте насколько уменьшилось число обучаемых параметров, как изменилась скорость во время forward и backward процессов и как изменились затраты по памяти. Сделайте выводы и напишите о них в отчете.

In [1]:
import time
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

/Users/sinfillo/Desktop/UniHW/NLP/HW4/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class LoRA(nn.Module):
    """
    Обёртка над nn.Linear с добавкой низкорангового сдвига BAx.
    Базовый слой W заморожен, обучаем только A и B.
    """
    def __init__(self, in_features, out_features, r, alpha, dropout, bias=True):
        super().__init__()

        self.in_features = in_features
        self.out_features = out_features
        self.r = r
        self.alpha = alpha
        self.scaling = alpha / r

        self.base = nn.Linear(in_features, out_features, bias=bias)

        self.lora_A = nn.Linear(in_features, r, bias=False)
        self.lora_B = nn.Linear(r, out_features, bias=False)

        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

        self.base.reset_parameters()
        nn.init.normal_(self.lora_A.weight, mean=0.0, std=0.02)
        nn.init.zeros_(self.lora_B.weight)

    def forward(self, x):
        base_out = self.base(x)
        lora_out = self.lora_B(self.lora_A(self.dropout(x))) * self.scaling
        return base_out + lora_out


In [3]:
def freeze_base_and_unfreeze_lora(model):
    for p in model.parameters():
        p.requires_grad = False

    for name, p in model.named_parameters():
        if "lora_A" in name or "lora_B" in name:
            p.requires_grad = True

In [4]:
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

In [5]:
def get_parent_module(model, module_name):
    parts = module_name.split(".")
    parent = model
    for p in parts[:-1]:
        parent = getattr(parent, p)
    return parent, parts[-1]


In [6]:
def apply_lora(model, r=8, alpha=16, dropout=0.05, target=("q_proj", "v_proj")):
    for name, module in list(model.named_modules()):
        if isinstance(module, nn.Linear) and any(t in name for t in target):
            parent, attr = get_parent_module(model, name)

            lora = LoRA(module.in_features, module.out_features, r=r, alpha=alpha, dropout=dropout, bias=(module.bias is not None))
            lora.base.weight.data = module.weight.data.clone()
            if module.bias is not None:
                lora.base.bias.data = module.bias.data.clone()

            setattr(parent, attr, lora)

In [7]:
class CausalLMDataset(Dataset):
    def __init__(self, token_ids, block_size):
        self.data = token_ids
        self.block_size = block_size

    def __len__(self):
        return (len(self.data) - 1) // self.block_size

    def __getitem__(self, idx):
        start = idx * self.block_size
        end = start + self.block_size
        x = self.data[start:end]
        y = self.data[start + 1:end + 1]
        return x, y

In [8]:
from datasets import load_dataset

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "Qwen/Qwen2-0.5B"
block_size = 128
batch_size = 8

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

raw = load_dataset("wikitext", "wikitext-2-raw-v1")
train_text = "\n\n".join(raw["train"]["text"])
valid_text = "\n\n".join(raw["validation"]["text"])

train_ids = tokenizer(train_text, return_tensors="pt", add_special_tokens=False)["input_ids"].squeeze(0)
valid_ids = tokenizer(valid_text, return_tensors="pt", add_special_tokens=False)["input_ids"].squeeze(0)

train_ds = CausalLMDataset(train_ids, block_size)
valid_ds = CausalLMDataset(valid_ids, block_size)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_ds, batch_size=batch_size)


Generating validation split: 100%|██████████| 3760/3760 [00:00<00:00, 1094039.75 examples/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (2518423 > 32768). Running this sequence through the model will result in indexing errors


In [10]:
@torch.no_grad()
def evaluate_lm_loss(model, dataloader, device):
    model.eval()
    accum_loss = 0.0
    n_tokens = 0
    for x, y in dataloader:
        x = x.to(device)
        y = y.to(device)
        out = model(input_ids=x, labels=y)
        loss = out.loss
        B, T = x.shape
        accum_loss += loss.item() * (B * T)
        n_tokens += B * T
    return accum_loss / n_tokens

def train_epoch_lm(model, dataloader, optimizer, device, clip=1.0):
    model.train()
    accum_loss = 0.0
    n_tokens = 0
    for x, y in dataloader:
        x = x.to(device)
        y = y.to(device)

        out = model(input_ids=x, labels=y)
        loss = out.loss

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        B, T = x.shape
        accum_loss += loss.item() * (B * T)
        n_tokens += B * T

    return accum_loss / n_tokens

In [ ]:
def run_experiment(model, train_loader, valid_loader, device, lr=1e-4, epochs=1, clip=1.0, desc=""):
    model.to(device)
    total, trainable = count_parameters(model)

    print(f"\n=== {desc} ===")
    print(f"Всего параметров:     {total/1e6:.2f}M")
    print(f"Обучаемых параметров: {trainable/1e6:.2f}M " f"({100*trainable/total:.3f}%)")

    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    history = []
    for epoch in range(1, epаochs + 1):
        if device.startswith("cuda"):
            torch.cuda.reset_peak_memory_stats(device)
        start = time.perf_counter()

        train_loss = train_epoch_lm(model, train_loader, optimizer, device, clip)
        val_loss = evaluate_lm_loss(model, valid_loader, device)

        elapsed = time.perf_counter() - start
        if device.startswith("cuda"):
            max_mem = torch.cuda.max_memory_allocated(device) / 1024**2
        else:
            max_mem = float("nan")

        history.append((train_loss, val_loss, elapsed, max_mem))

        print(f"epoch {epoch}: train_loss={train_loss:.4f}, "
              f"val_loss={val_loss:.4f}, "
              f"time={elapsed:.2f}s, "
              f"max_gpu_mem={max_mem:.1f} MB")

    return {
        "description": desc,
        "total_params": total,
        "trainable_params": trainable,
        "history": history,
    }

In [ ]:
def make_short_loader(loader, max_batches=200):
    it = iter(loader)
    for _ in range(max_batches):
        try:
            yield next(it)
        except StopIteration:
            return

base_model = AutoModelForCausalLM.from_pretrained(model_name)
result_full = run_experiment(base_model, make_short_loader(train_loader, max_batches=200), make_short_loader(valid_loader, max_batches=50), device=device, lr=1e-4, epochs=1, desc="Qwen full fine-tune")


=== Qwen full fine-tune ===
Всего параметров:     494.03M
Обучаемых параметров: 494.03M (100.000%)


In [ ]:
lora_model = AutoModelForCausalLM.from_pretrained(model_name)

apply_lora(lora_model, r=8, alpha=16, dropout=0.05)
freeze_base_and_unfreeze_lora(lora_model)

result_lora = run_experiment(lora_model, make_short_loader(train_loader, max_batches=200), make_short_loader(valid_loader, max_batches=50), device=device, lr=1e-3, epochs=1, desc="Qwen + LoRA (только A,B)")

### Supervised Fine-tuning

__Задание 2 (3 балла).__ Разбейте все примеры с "хорошими" ответами на запросы (все что идет до последнего "Assistant:") и ответы (все, начиная с последнего "Assistant:"). Дообучите модель [`pythia-1.4b`](https://huggingface.co/EleutherAI/pythia-1.4b) генерировать правильные ответы с помощью вашей LoRA. Одной эпохи вполне должно хватить для сходимости. Проверьте на нескольких случайных тестовых примерах, что модель ведет себя так, как надо.

In [ ]:
# your code here

### Direct Preference Optimization

__Задание 3 (3 балла).__ Реализуйте DPO согласно [статье](https://arxiv.org/pdf/2305.18290) и дообучите SFT модель с предыдущего шага. Одной эпохи так же должно хватить, но можно обучать и дольше. Убедитесь, что модель начинает отдавать предпочтение хорошим ответам. Проведите анализ. Стали ли ответы лучше, чем у SFT модели? Всегда ли модель отвечает хорошо или иногда плохо? Насколько легко модель ломается при изменении промптов?